In [1]:
import os
import time
from pathlib import Path

import numpy as np
import tomli
from generate_data import sample_init_state

from models import ModelConfig, get_model_class
from system import SimulationConfig, simulate_system
from unicycle import Unicycle

base_path = Path(os.getcwd()).parent

Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.


Params

In [2]:
config = 'configs/config.toml'
abs_config_path = (base_path / config).resolve()
with open(abs_config_path, 'rb') as f:
    data = tomli.load(f)
    simulation_cfg = SimulationConfig(**data['simulation'])

Evaluation of computation time for single prediction

In [3]:
model_names = ['Numerical', 'FNO', 'DeepONet']
time_steps = [0.01, 0.005, 0.001, 0.0005]

perf_table = np.zeros((len(model_names), len(time_steps)))
model_params = dict()

predictor_trials = 100000
for i, model_name in enumerate(model_names):
    for j, time_step in enumerate(time_steps):
        simulation_cfg.dt = time_step

        # Generate random initial conditions
        init_state = np.ones(3, dtype=np.float32)
        init_inputs = [np.zeros(NDi, dtype=np.float32) for NDi in simulation_cfg.NDs]
        ml_unicycle = Unicycle(init_state, init_inputs, simulation_cfg)
        if model_name != 'Numerical':
            ml_unicycle.ml_predictor_model = get_model_class(model_name)(
                n_states=3,
                m_inputs=2,
                num_points=simulation_cfg.NDs[-1],
                dt=simulation_cfg.dt,
                delays=simulation_cfg.delays,
            ).to('cuda')
            ml_unicycle.use_only_ml_predictor()
            model_params[model_name] = ml_unicycle.get_model_params()

        # Evaluate predictor
        start_time = time.time()
        for k in range(predictor_trials):
            if ml_unicycle.predict_exact:
                ml_unicycle.predictor(t0=0)
            if ml_unicycle.predict_ml and ml_unicycle.ml_predictor_model is not None:
                ml_unicycle.ml_predictor()
        end_time = time.time()
        perf_table[i, j] = (end_time - start_time) / predictor_trials * 1000


# Print perf table
print("RAW CALCULATION TIMES (ms)")

header = f"{'Step size':>15}" + "".join([f"{ts:>12.4f}" for ts in time_steps])
print(header)
print("-" * len(header))

# Print each row
for i, name in enumerate(model_names):
    print(f"{name:>15}" + "".join([f"{perf:12.4f}" for perf in perf_table[i]]))

RAW CALCULATION TIMES (ms)
      Step size      0.0100      0.0050      0.0010      0.0005
---------------------------------------------------------------
      Numerical      0.3817      0.7528      3.8328      7.6564
            FNO      3.9267      3.9208      4.2176      4.0275
       DeepONet      1.0821      1.0492      1.5489      3.8348


Evaluation of computation time for trajectory

In [4]:
model_names = ['Numerical', 'FNO', 'DeepONet']
time_steps = [0.01, 0.005, 0.001]

perf_table = np.zeros((len(model_names), len(time_steps)))
l2_norm_table = np.zeros((len(model_names), len(time_steps)))
model_params = {
    model_name: np.zeros(len(time_steps), dtype=np.int32)
    for model_name in model_names
}

num_trajectories = 5
for i, model_name in enumerate(model_names):
    for j, time_step in enumerate(time_steps):
        simulation_cfg.dt = time_step
        
        # Construct model config
        model_cfg = None
        if model_name != 'Numerical':
            D1, D2 = simulation_cfg.delays
            time_step_str = f'{time_step}'.split('.')[1]
            model_path = (base_path / f'models/UnicycleConstDistinctDelays_{model_name}_{int(D1 * 100)}_{int(D2 * 100)}_dt_{time_step_str}.pth').resolve()
            model_cfg = ModelConfig(model_name, model_path)

        # Evaluate model performance
        total_time = 0
        total_trajectory_error = 0
        for _ in range(num_trajectories):
            
            # Get new initial condition for each trajectory
            init_state = sample_init_state()
            init_inputs = [np.zeros(NDi, dtype=np.float32) for NDi in simulation_cfg.NDs]

            # Simulation
            ml_unicycle = Unicycle(init_state, init_inputs, simulation_cfg, model_cfg)
            if model_cfg is not None:
                ml_unicycle.use_only_ml_predictor()
                model_params[model_name][j] = ml_unicycle.get_model_params()

            # Computation time
            start_time = time.time()
            states, _, _, _ = simulate_system(ml_unicycle)
            end_time = time.time()

            total_time += (end_time - start_time)

            # L2 trajectory error
            squared_magnitudes = np.sum(states**2, axis=1)
            total_trajectory_error += np.sqrt(np.trapezoid(squared_magnitudes, dx=time_step))

        # Average
        perf_table[i, j] = total_time / num_trajectories
        l2_norm_table[i, j] = total_trajectory_error / num_trajectories


# Model parameters
print("PARAMETERS")
for name, data in model_params.items():
    print(f"{name:>15}" + "".join([f"{number:>12d}" for number in data]))


# Print perf table
print("\nRAW CALCULATION TIMES (sec)")

header = f"{'Step size':>15}" + "".join([f"{ts:>12.4f}" for ts in time_steps])
print(header)
print("-" * len(header))

# Print each row
for i, name in enumerate(model_names):
    print(f"{name:>15}" + "".join([f"{perf:12.4f}" for perf in perf_table[i]]))


# Print L2 trajectory error
print('\nAVERAGE L2 TRAJECTORY ERROR')

header = f"{'Step size':>15}" + "".join([f"{ts:>12.4f}" for ts in time_steps])
print(header)
print("-" * len(header))

for i, name in enumerate(model_names):
    print(f"{name:>15}" + "".join([f"{error:12.4f}" for error in l2_norm_table[i]]))

/home/filip/UCSD/Unicycle/src/system.py:165: RuntimeWarning: overflow encountered in cast
  controls[i-1] = system.controller(t[i-1])
/tmp/ipykernel_2746258/1824678377.py:47: RuntimeWarning: overflow encountered in square
  squared_magnitudes = np.sum(states**2, axis=1)


PARAMETERS
      Numerical           0           0           0
            FNO       42147       42147       42147
       DeepONet      358309     1405609    34667209

RAW CALCULATION TIMES (sec)
      Step size      0.0100      0.0050      0.0010
---------------------------------------------------
      Numerical      0.5468      1.8395     38.5755
            FNO      3.8828      7.6460     40.8908
       DeepONet      1.1529      2.3020     17.0408

AVERAGE L2 TRAJECTORY ERROR
      Step size      0.0100      0.0050      0.0010
---------------------------------------------------
      Numerical      2.0237      2.0201      2.1874
            FNO         nan      1.9174      1.9821
       DeepONet      2.1224      1.9711      2.0149
